# Experiment 2: Interactive reprocessing
This is representative of a new type of functionality that we currently don’t support in METASPACE
because it’s uneconomical with the serverful approach. While looking for specific compounds,
scientists tend to have relatively short lists of molecules of interest, and iteratively try
different adducts or modifiers until they find the data they’re interested in.

### METRICS TO BENCHMARK
* Performance:
    * **Metric:** Total processing time
    
        **Goal:** Fast enough to use interactively in a notebook - less than ~60 seconds

* Cost:
    * **Metric:** Total cost
    
        **Goal:** Significantly less than a full annotation - determined by experiment 1

# Notebook setup
Run `python3 setup.py install` to install all requirements for annotation pipeline project.

In [ ]:
# We need this to overcome Python notebooks limitations of too many open files
import resource
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('Before:', soft, hard)

# Raising the soft limit. Hard limits can be raised only by sudo users
resource.setrlimit(resource.RLIMIT_NOFILE, (10000, hard))
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print('After:', soft, hard)

In [ ]:
%config Completer.use_jedi = False
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# If pywren_ibm_cloud isn't installed, please run `pip install -e .` in this directory to install all dependencies
import pywren_ibm_cloud as pywren

pywren.__version__

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
# Set a socket timeout so that CF requests fail instead of hanging if they don't get a response
import socket
print('Previous timeout:', socket.getdefaulttimeout())
socket.setdefaulttimeout(60)

## Configuration

In [ ]:
import json
config = json.load(open('config.json'))

In [ ]:
# Input dataset
input_ds = json.load(open('metabolomics/ds_config2.json'))
# Input database (Used as a template. Some parameters overridden below...)
input_db = json.load(open('metabolomics/db_config2.json'))

# Override databases, because this experiment expects a small database
exp_db_path = 'metabolomics/db/mol_db5.pickle'
input_db['databases'] = [exp_db_path]

# Initial setup (not included in benchmark timings)

In [ ]:
from annotation_pipeline.molecular_db import upload_mol_db_from_file
from annotation_pipeline.pipeline import Pipeline
from datetime import datetime


### Load & segment dataset

In [ ]:
pipeline = Pipeline(config, input_ds, input_db, use_cache=False)
pipeline.load_ds()
pipeline.split_ds()
pipeline.segment_ds()

# Benchmark

In [ ]:
from annotation_pipeline.utils import get_ibm_cos_client
cos_client = get_ibm_cos_client(config)

### Process new molecules and Run Annotation

In [ ]:
start_time = datetime.now()

# Process new molecules:
## Upload list of molecules (in a real scenario this list would change every iteration, so this isn't part of setup)
upload_mol_db_from_file(config, config['storage']['db_bucket'], exp_db_path, 'metabolomics/db/mol_db5.csv')
pipeline.build_database()
pipeline.calculate_centroids()
pipeline.segment_centroids()

# Run Annotation:
pipeline.annotate()
results_df = pipeline.formula_metrics_df

finish_time = datetime.now()

In [ ]:
print('start', start_time)
print('finish', finish_time)
print('duration', finish_time - start_time)

In [ ]:
# Display PyWren statistics file
from annotation_pipeline.utils import get_pywren_stats
get_pywren_stats()

In [ ]:
# Display results
print(results_df.shape)
results_df.head()

# Clean Temp Data

In [ ]:
pipeline.clean()